In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np 
import torch
import scipy
import scipy.sparse
from math import prod
from math import sqrt, floor, ceil
import pandas as pd


In [10]:
t = np.zeros((set_size+1, set_size+1), dtype=np.int32)
for i in range(1, set_size+1):
    for j in range(1, set_size+1):
        t[i, j] = (i)*(j)
r = np.zeros((set_size+1, set_size+1), dtype=np.int32)
for i in range(1, set_size+1):
    for j in range(1, set_size+1):
        if i!=j and sqrt(t[i,j]) == floor(sqrt(t[i,j])):
            r[i,j] = 1
i, j = np.where(np.triu(r[1:, 1:])==1)
invalid_combos = [(a+1,b+1) for a, b in zip(i.tolist(), j.tolist())]
invalid_combos


[(1, 4), (1, 9), (2, 8), (4, 9)]

In [49]:
set_size = 500
r = np.zeros((set_size+1, set_size+1), dtype=np.int32)
for i in range(1, set_size+1):
    for j in range(1, set_size+1):
        if i!=j and sqrt((i)*(j)) == floor(sqrt((i)*(j))):
            r[i,j] = 1
a, b = np.where(np.triu(r[1:, 1:])==1)
m = gp.Model("find")
X = m.addVars(set_size, vtype=GRB.BINARY, name="X")
m.setObjective(
    gp.quicksum(
        X[i]
        for i in range(len(X))
    ),
    GRB.MAXIMIZE,
)
m.addConstrs(
    (X[i] <= 1-X[j] for i, j in zip(a,b)),
    name="a_or_b_or_neither_constraint",
)
m.optimize()

# Sanity Check
sol = [idx+1 for idx, v in enumerate(m.getVars()) if v.X == 1]
for i in sol:
    for j in sol:
        if i!=j and sqrt((i)*(j)) == floor(sqrt((i)*(j))):
            print('fail')

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 729 rows, 500 columns and 1458 nonzeros
Model fingerprint: 0xcdb58cd7
Variable types: 0 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 306.0000000
Presolve removed 729 rows and 500 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 306 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.060000000000e+02, best bound 3.060000000000e+02, gap 0.0000%


In [2]:
def generate_01_sparse_matrix(size, density):
    f, c, ky, kx = size
    x = scipy.sparse.rand(f, c*ky*kx, density, format='csr')
    x.data[:] = 1
    x = np.array(x.todense()).reshape((f, c, ky, kx))
    return x

In [3]:
def get_ones_count(matrix):
    return prod(matrix[np.where(matrix == 1)].shape)

In [4]:
def calc_density(matrix):
    ones = get_ones_count(matrix)
    return ones/prod(matrix.shape)

In [5]:
def density_of_remaining_weights(original_matrix, selection_bitmap):
    bitmap = np.zeros(original_matrix.shape[0:2])
    for i, j in selection_bitmap:
        bitmap[i, j] = 1
    not_selected = 1-bitmap
    total_unselected = np.sum(not_selected)
    ignored_weights = np.multiply(not_selected, np.sum(original_matrix, axis=(3, 2)))
    unselected_ones = get_ones_count(ignored_weights)
    return  unselected_ones/ total_unselected 

In [6]:
weight_tensor = generate_01_sparse_matrix(size = (64, 64, 3, 3), density = 0.70)
weight_tensor[np.where(weight_tensor != 0)]

array([1., 1., 1., ..., 1., 1., 1.])

In [7]:
def find_densest_subtensor_in_weight_tensor(
    tensor,
    min_filters=None,
    min_channels=None,
    initialize=False,
    timeout=None,
    avoid_bitmap=None,
):
    f_size, c_size, ky_size, kx_size = tensor.shape
    if min_filters is not None and min_filters > f_size:
        raise ValueError("filter lowerbound must be lower than max filters")
    if min_channels is not None and min_channels > c_size:
        raise ValueError("channel lowerbound must be lower than max filters")

    tensor_cpy = np.copy(tensor)
    tensor_cpy[np.where(tensor_cpy == 0)] = -1
    m = gp.Model("densify")
    if timeout is not None:
        m.setParam(GRB.Param.TimeLimit, timeout)
    F = m.addVars(f_size, vtype=GRB.BINARY, name="F")
    C = m.addVars(c_size, vtype=GRB.BINARY, name="C")

    if initialize:
        est_filter_density = [
            (i, s) for s, i in zip(np.sum(tensor, axis=(2, 1)), range(tensor.shape[0]))
        ]
        est_filter_density.sort(key=lambda x: x[1], reverse=True)
        est_channel_density = [
            (i, s) for s, i in zip(np.sum(tensor, axis=(2, 0)), range(tensor.shape[1]))
        ]
        est_channel_density.sort(key=lambda x: x[1], reverse=True)
        initial_filters = [i for i, _ in est_filter_density[: min_filters + 1]]
        initial_channels = [j for j, _ in est_channel_density[: min_channels + 1]]

        for i in initial_filters:
            F[i].start = 1
        for j in initial_channels:
            C[j].start = 1

    Z = m.addVars(f_size, c_size, vtype=GRB.BINARY, name="Z")

    if avoid_bitmap is not None:
        m.addConstrs(
            (Z[i, j] == 0 for i, j in avoid_bitmap),
            name="avoid_constraints",
        )

    if min_filters is not None:
        m.addConstr(min_filters == gp.quicksum([F[i] for i in range(len(F))]))
    if min_channels is not None:
        m.addConstr(gp.quicksum([C[j] for j in range(len(C))]) <= min_channels)
        
    m.addConstrs(
        (Z[i, j] == gp.and_(F[i], C[j]) for i in range(len(F)) for j in range(len(C))),
        name="and_constraints",
    )
    m.setObjective(
        gp.quicksum(
            Z[i, j] * tensor_cpy[i, j, ky, kx]
            for i in range(len(F))
            for j in range(len(C))
            for ky in range(ky_size)
            for kx in range(kx_size)
        ),
        GRB.MAXIMIZE,
    )
    m.optimize()
    dense_filter_indicies = [i for i, f in F.items() if f.X > 0]
    dense_channel_indicies = [j for j, c in C.items() if c.X > 0]
    selection_bitmap = [
        (i, j) for i in range(len(F)) for j in range(len(C)) if Z[i, j].X == 1
    ]
    dense_tensor = tensor[dense_filter_indicies, :, :][:, dense_channel_indicies, :]
    return dense_tensor, selection_bitmap, dense_filter_indicies, dense_channel_indicies


In [8]:
input_tensor = weight_tensor
size = prod(input_tensor.shape[0:2])
density_tracker = [calc_density(input_tensor)]
target_size = 8
selection_bitmap = None
for i in range(size//target_size**2):
    initial_size = input_tensor.shape[0]
    print(f'Reducing from tensor of size {initial_size} to tensors of size {target_size}')
    (
        output_tensor,
        selection_bitmap,
        dense_filter_indicies,
        dense_channel_indicies,
    ) = find_densest_subtensor_in_weight_tensor(input_tensor, target_size, target_size, timeout=25, avoid_bitmap=selection_bitmap)
    input_density = calc_density(input_tensor)
    output_density = calc_density(output_tensor)
    density_tracker.append(output_density)
    remaining_density = density_of_remaining_weights(input_tensor, selection_bitmap)
    
    print(f"density of input tensor: {input_density}")
    print(f"density of output tensor: {output_density}")
    print(
        f"density of sparse tensor: {remaining_density}"
    )
    print(f"selected filters: {dense_filter_indicies}")
    print(f"selected channels: {dense_channel_indicies}")
    input_tensor = output_tensor


Reducing from tensor of size 64 to tensors of size 8
Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-04
Set parameter TimeLimit to value 25
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4224 columns and 128 nonzeros
Model fingerprint: 0xca30eb92
Model has 4096 general constraints
Variable types: 0 continuous, 4224 integer (4224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]
Presolve added 7784 rows and 0 columns
Presolve time: 0.03s
Presolved: 7786 rows, 4224 columns, 16104 nonzeros
Variable types: 0 continuous, 4224 integer (4224 binary)
Found heuristic solution: objective 312.0000000

Root relaxation: objective 1.926500e+03, 5043 iterations, 0.09 seconds (0.10 work units)

    Nodes    |    Current Node    |   

KeyError: (1, 32)